In [9]:
from forecast import get_forecast_data
from consumption import get_consumption_data
from production import get_production_data

In [11]:
start_date = "2019-01-01T00:00:00+01:00"
end_date = "2024-01-01T00:00:00+01:00"

df_forecast = get_forecast_data(start_date, end_date)
df_consumption = get_consumption_data(start_date, end_date)
df_production = get_production_data(start_date, end_date)

In [ ]:
# !pip install pandas sqlalchemy mysql-connector-python
# !pip install pymysql

In [10]:
from sqlalchemy import inspect, create_engine

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user="admin",
                               pw="admin2024",
                               host="mysql-database.c9cwko0aaypr.eu-central-1.rds.amazonaws.com",
                               db="rte"))

# Use the 'inspect' function to get the inspector for the engine
inspector = inspect(engine)

# Use the 'get_table_names' method to get the list of table names
table_names = inspector.get_table_names()

# Print the table names
for table_name in table_names:
    print(table_name)

consumption
consumption_and_forecast_5years
power_consumption
power_consumption_2019_2024
power_consumption_5years
production_consumption_and_forecast_5years
production_type
production_type_2019_2024
production_type_5years
test
weekly_forecast
weekly_forecast_2019_2024
weekly_forecast_5years


In [12]:
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.sql import text
import pandas as pd

# Assuming df1, df2, df3 are your dataframes
# Replace user, password, host, dbname with your actual values
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user="admin",
                               pw="admin2024",
                               host="mysql-database.c9cwko0aaypr.eu-central-1.rds.amazonaws.com",
                               db="rte"))

# Upload dataframes to MySQL
df_forecast.to_sql('forecast', con=engine, if_exists='replace', index=False)
df_consumption.to_sql('consumption', con=engine, if_exists='replace', index=False)
df_production.to_sql('production', con=engine, if_exists='replace', index=False)


522616

In [ ]:

# Define primary keys and relations
metadata = MetaData()

table1 = Table('table1', metadata, autoload_with=engine)
table2 = Table('table2', metadata, autoload_with=engine)
table3 = Table('table3', metadata, autoload_with=engine)

# Assuming 'id' is the primary key for table1 and table2, and 'table1_id' and 'table2_id' are the foreign keys in table3
with engine.connect() as connection:
    connection.execute(text("ALTER TABLE table1 ADD PRIMARY KEY (id);"))
    connection.execute(text("ALTER TABLE table2 ADD PRIMARY KEY (id);"))
    connection.execute(text("ALTER TABLE table3 ADD FOREIGN KEY (table1_id) REFERENCES table1(id);"))
    connection.execute(text("ALTER TABLE table3 ADD FOREIGN KEY (table2_id) REFERENCES table2(id);"))

In [ ]:
with engine.connect() as connection:
    connection.execute(text("ALTER TABLE table_name ADD PRIMARY KEY (column1, column2);"))

In [ ]:
with engine.connect() as connection:
    # Assuming 'consumption_start_date', 'production_start_date', 'forecast_start_date' are the primary keys
    connection.execute(text("ALTER TABLE consumption ADD PRIMARY KEY (consumption_start_date);"))
    connection.execute(text("ALTER TABLE production ADD PRIMARY KEY (production_start_date);"))
    connection.execute(text("ALTER TABLE forecast ADD PRIMARY KEY (forecast_start_date);"))

    # Add foreign keys in 'production' and 'forecast' that reference 'consumption'
    connection.execute(text("ALTER TABLE production ADD FOREIGN KEY (consumption_start_date) REFERENCES consumption(consumption_start_date);"))
    connection.execute(text("ALTER TABLE forecast ADD FOREIGN KEY (consumption_start_date) REFERENCES consumption(consumption_start_date);"))

In [ ]:
# Upload dataframes to MySQL
production.to_sql('production', con=engine, if_exists='replace', index=False)
consumption.to_sql('consumption', con=engine, if_exists='replace', index=False)
forecast.to_sql('forecast', con=engine, if_exists='replace', index=False)

# Define primary keys and relations
metadata = MetaData()

production = Table('production', metadata, autoload_with=engine)
consumption = Table('consumption', metadata, autoload_with=engine)
forecast = Table('forecast', metadata, autoload_with=engine)

# Assuming 'consumption_start_date' is the primary key for consumption, 
# and 'production_start_date' and 'forecast_start_date' are the foreign keys in production and forecast respectively
with engine.connect() as connection:
    connection.execute(text("ALTER TABLE consumption ADD PRIMARY KEY (consumption_start_date);"))
    connection.execute(text("ALTER TABLE production ADD FOREIGN KEY (production_start_date) REFERENCES consumption(consumption_start_date);"))
    connection.execute(text("ALTER TABLE forecast ADD FOREIGN KEY (forecast_start_date) REFERENCES consumption(consumption_start_date);"))